In [1]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from distutils.util import strtobool
from keras import backend
from einops import rearrange
import time

# from data_provider.data_factory import data_provider
# from ..utils.metrics import smape

def smape(y_true, y_pred):
    """ Calculate Armstrong's original definition of sMAPE between `y_true` & `y_pred`.
        `loss = 200 * mean(abs((y_true - y_pred) / (y_true + y_pred), axis=-1)`
        Args:
        y_true: Ground truth values. shape = `[batch_size, d0, .. dN]`.
        y_pred: The predicted values. shape = `[batch_size, d0, .. dN]`.
        Returns:
        Symmetric mean absolute percentage error values. shape = `[batch_size, d0, ..
        dN-1]`.
        """
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    diff = tf.abs(
        (y_true - y_pred) /
        backend.maximum(y_true + y_pred, backend.epsilon())
    )
    return 200.0 * backend.mean(diff, axis=-1)

def smapeofa(true, pred, agg=np.mean):
    smape = 200 * np.abs(pred - true) / (np.abs(pred) + np.abs(true) + 1e-8)
    if agg is None:
        return smape
    else:
        return agg(smape)
    
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )

class M4ZeroShotDataset(Dataset):   
    def __init__(self, train, horizon, history, test=None, max_size=500):
        self.make_test = False
        if test is None:
            self.make_test = True
        else:
            self.test = self.load_data(test)

        self.train = self.load_data(train)

        self.horizon = horizon
        self.history = history
        self.max_size = max_size


    def load_data(self, data):
        if isinstance(data, list):
            timeseries = data
        else:
            if isinstance(data, str):
                if data.split('.')[-1] == 'tsf':
                    data = convert_tsf_to_dataframe(data)[0]
                    timeseries = [self.dropna(ts).astype(np.float32) for ts in data.loc[:, 'series_value'].values]
                else:
                    data = pd.read_csv(data) 
                    timeseries = [self.dropna(ts).values.astype(np.float32) for n, ts in data.iloc[:, 1:].iterrows()]
        return timeseries


    @staticmethod
    def dropna(x):
        return x[~np.isnan(x)]

    def __len__(self):
        return len(self.train)

    def __getitem__(self, index):

        series_train = self.train[index]
        if self.make_test:
            series_test = series_train[-self.horizon:]
            series_train = series_train[:-self.horizon]
        else:
            series_test = self.test[index]

        len_train = np.minimum(self.history, len(series_train))
        pad_size = self.max_size - len_train

        x_train = series_train[-len_train:]
        x_val = series_test

        train_min = np.min(x_train)
        train_ptp = np.ptp(x_train)
        if np.isclose(train_ptp, 0):
            train_ptp = 1
        x_train = (x_train - train_min) / train_ptp
        x_val = (x_val - train_min) / train_ptp
        x_train = x_train.astype(np.float32)
        x_val = x_val.astype(np.float32)

        mask_train = np.pad(np.ones(len(x_train)), (0, pad_size), mode = 'constant')
        x_train = np.pad(x_train, (0, pad_size), mode = 'constant')
        
        
        out = {
            'series': x_train.astype('float32'),
            'target': x_val.astype('float32'),
            'mask_series': mask_train,
            'min': train_min,
            'ptp': train_ptp,
        }
        return out
    
def sliding_window_view(a, window):
    """Generate window view."""
    shape = a.shape[:-1] + (a.shape[-1] - window + 1,) + (window,)
    strides = a.strides[:-1] + (a.strides[-1],) + a.strides[-1:]
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def rolling_window(a, window, step=1, from_last=True):
    """from_last == True - will cut first step-1 elements"""
    sliding_window = (
        sliding_window_view(a, window)
        if np.__version__ < "1.20"
        else np.lib.stride_tricks.sliding_window_view(a, window)
    )
    return sliding_window[(len(a) - window) % step if from_last else 0 :][::step]


class TopInd:
    def __init__(
        self, n_target=7, history=100, step=3, from_last=True, test_last=True, date_col=None, scheme=None, **kwargs
    ):
        self.n_target = n_target
        self.history = history
        self.step = step
        self.from_last = from_last
        self.test_last = test_last
        self.date_col = date_col

    @staticmethod
    def _timedelta(x):
        delta = pd.to_datetime(x).diff().iloc[-1]
        if delta <= pd.Timedelta(days=1):
            return pd.to_datetime(x).diff().fillna(delta).values, delta

        if delta > pd.Timedelta(days=360):
            d = pd.to_datetime(x).dt.year.diff()
            delta = d.iloc[-1]
            return d.fillna(delta).values, delta
        elif delta > pd.Timedelta(days=27):
            d = pd.to_datetime(x).dt.month.diff() + 12 * pd.to_datetime(x).dt.year.diff()
            delta = d.iloc[-1]
            return d.fillna(delta).values, delta
        else:
            return pd.to_datetime(x).diff().fillna(delta).values, delta

    def read(self, data, plain_data=None):
        self.len_data = len(data)
        self.date_col = self.date_col
        self.time_delta = self._timedelta(data[self.date_col])[1]
        return self
        # TODO: add asserts

    def _create_test(self, data=None, plain_data=None):
        # for predicting future
        return rolling_window(
            np.arange(self.len_data if data is None else len(data)), self.history, self.step, self.from_last
        )[-1 if self.test_last else 0 :, :]

    def _create_data(self, data=None, plain_data=None):

        return rolling_window(
            np.arange(self.len_data if data is None else len(data))[: -self.n_target],
            self.history,
            self.step,
            self.from_last,
        )

    def _create_target(self, data=None, plain_data=None):
        return rolling_window(
            np.arange(self.len_data if data is None else len(data))[self.history :],
            self.n_target,
            self.step,
            self.from_last,
        )

    def _get_ids(self, data=None, plain_data=None, func=None, cond=None):
        date_col = pd.to_datetime(data[self.date_col])
        vals, time_delta = self._timedelta(data[self.date_col])
        ids = list(np.argwhere(vals != time_delta).flatten())
        prev = 0
        inds = []
        for split in ids + [len(date_col)]:
            segment = date_col.iloc[prev:split]
            if len(segment) > cond:
                ind = func(segment) + prev
                inds.append(ind)
            prev = split
        inds = np.vstack(inds)
        return inds

    def create_data(self, data=None, plain_data=None):
        return self._get_ids(data, plain_data, self._create_data, self.n_target + self.history)

    def create_test(self, data=None, plain_data=None):
        return self._get_ids(data, plain_data, self._create_test, self.history)

    def create_target(self, data=None, plain_data=None):
        return self._get_ids(data, plain_data, self._create_target, self.n_target + self.history)

class BenchZeroShotDataset(Dataset):   
    def __init__(self, horizon, history, real_size, train, borders, max_size=500, date_column = 'date', scale=True):

        self.horizon = horizon
        self.history = history
        self.max_size = max_size

        self.real_size = real_size
        self.borders = borders

        self.date_column = date_column
        self.scale = scale
        self.scaler = None
        self.targets = None

        train, val, test = self.load_data(train)

        L_split_data = val[date_column].values[(len(val) - history) if (len(val) - history) > 0 else 0]
        L_last_val_data = val[val[date_column] >= L_split_data]
        if (len(val) - history) < 0:
            L_split_data = train[date_column].values[(len(train) - (history- len(L_last_val_data))) if (len(train) - (history- len(L_last_val_data))) > 0 else 0]
            L_last_train_data = train[train[date_column] >= L_split_data]
            test_data_expanded = pd.concat((L_last_train_data, L_last_val_data, test))
        else:
            test_data_expanded = pd.concat((L_last_val_data, test))
        test_data_expanded = test_data_expanded.sort_values([date_column]).reset_index(
            drop=True
        )
        self.data = test_data_expanded
#         print(self.data)
        slicer = TopInd(n_target=horizon, history=history, step=1, from_last=False, test_last=False, date_col=date_column)
        slicer.read(self.data)

        self.ids = slicer.create_data(self.data)
        self.ids_y = slicer.create_target(self.data)
        
        self.ts = self._ForecastPFN_time_features(self.data[self.date_column].values)
#         print(self.ts)
        self.data = self.data.loc[:, self.targets].values
        
        # add time features
        # self.ts = self.data.loc[:, self.data_stamp].values
        # self.ts = self._ForecastPFN_time_features(self.data.loc[:, self.data_stamp].values)
        # self.data_stamp_original = df_raw[border1:border2]
        # test_data.data_stamp = self._ForecastPFN_time_features(list(test_data.data_stamp_original['date']))
        
    def _ForecastPFN_time_features(self, ts: np.ndarray):
        if type(ts[0]) == datetime:
            year = [x.year for x in ts]
            month = [x.month for x in ts]
            day = [x.day for x in ts]
            day_of_week = [x.weekday()+1 for x in ts]
            day_of_year = [x.timetuple().tm_yday for x in ts]
            return np.stack([year, month, day, day_of_week, day_of_year], axis=-1)
        ts = pd.to_datetime(ts)
        return np.stack([ts.year, ts.month, ts.day, ts.day_of_week + 1, ts.day_of_year], axis=-1)


    def load_data(self, data):
        data = pd.read_csv(data) #[[self.date_column, 'OT']]
        
        self.targets = list(data.columns.drop(self.date_column))

        if self.borders is None:
            TEST_SIZE = 0.2
            VAL_SIZE = 0.1

            train_val_split_data = data[self.date_column].values[
                int(data[self.date_column].nunique() * (1 - (VAL_SIZE + TEST_SIZE)))
            ]
            val_test_slit_data = data[self.date_column].values[
                int(data[self.date_column].nunique() * (1 - TEST_SIZE))
            ]

            train = data[data[self.date_column] <= train_val_split_data]
            val = data[
                (data[self.date_column] > train_val_split_data) & (data[self.date_column] <= val_test_slit_data)
            ]
            test = data[data[self.date_column] > val_test_slit_data]
#             print(train_val_split_data)
#             print(val_test_slit_data)

        else:
            train = data.iloc[:self.borders[0]]
            val = data.iloc[self.borders[0]:self.borders[1]]
            test = data.iloc[self.borders[1]:self.borders[2]]

        if self.scale:
            self.scaler = StandardScaler()
            self.scaler.fit(train.loc[:, self.targets].values)
            train.loc[:, self.targets] = self.scaler.transform(train.loc[:, self.targets].values)
            val.loc[:, self.targets] = self.scaler.transform(val.loc[:, self.targets].values)
            test.loc[:, self.targets] = self.scaler.transform(test.loc[:, self.targets].values)
        
#         self.train_ts = self._ForecastPFN_time_features(list(train[self.date_column]))
#         self.val_ts = self._ForecastPFN_time_features(list(val[self.date_column]))
#         self.test_ts = self._ForecastPFN_time_features(list(test[self.date_column]))
#         self.train_ts = self._ForecastPFN_time_features(list(train[self.date_column]))
#         self.test_ts = self._ForecastPFN_time_features(list(test[self.date_column]))
        
#         self.ts = self._ForecastPFN_time_features(list(data[self.date_column]))
#         self.
#         self.ts = list(data[self.date_column])
#         self.val_ts = val.loc[:, self.date_column].values
#         self.test_ts = test.loc[:, self.date_column].values
        

        return train, val, test

    def __len__(self):
        return int(len(self.ids) * len(self.targets))

    def __getitem__(self, index):
        #index = index + 1
        ind, target = index // len(self.targets), index % len(self.targets)
#         print(1, ind)
#         print(2, target)
        
        series_train = self.data[self.ids[ind], target]
#         print(self.ids[ind])
#         print(3, series_train.shape)
        
        series_test = self.data[self.ids_y[ind], target]
#         print(self.ids_y[ind])
#         print(4, series_test.shape)
        
        seq_x_mark = self.ts[self.ids[ind]]
#         print(5, seq_x_mark.shape)
#         seq_x_mark = self.val_ts
#         print(seq_x_mark)
        
        seq_y_mark = self.ts[self.ids_y[ind]]
#         print(6, seq_y_mark.shape)
#         seq_y_mark = self.test_ts
#         print(seq_y_mark)

        len_train = np.minimum(self.real_size, len(series_train))
#         print(7, len_train)
#         print(len(series_train))
        
        pad_size = self.max_size - len_train
#         print(8, pad_size.shape)

        x_train = series_train[-len_train:].astype(np.float32)
#         print(9, x_train.shape)
        
        x_test = series_test.astype(np.float32)
#         print(10, x_val.shape)
        
        seq_x_mark = seq_x_mark.astype(np.float32)
#         print(11, seq_x_mark.shape)
        
        seq_y_mark = np.expand_dims(seq_y_mark.astype(np.float32), axis=1)
#         print(12, seq_y_mark.shape)
        
        if np.all(x_train == x_train[0]):
            x_train[-1] += 1
        x_train = np.expand_dims(x_train, axis=1)
#         print(13, x_train.shape)
#         print(x_train)
        
        history = x_train.copy()
#         print(14, history.shape)
        
        mean = np.nanmean(history)
#         print(15, mean)
        
        std = np.nanstd(history)
#         print(16, std)
        
        history = (history - mean) / std
#         print(17, history.shape)
        
        history_mean = np.nanmean(history[-6:])
#         print(18, history_mean)
        
        history_std = np.nanstd(history[-6:])
#         print(19, history_std)
        
        local_scale = (history_mean + history_std + 1e-4)
#         print(20, local_scale)
        
        history = np.clip(history / local_scale, a_min=0, a_max=1)
#         print(21, history.shape)
#         print(history)
        
        if x_train.shape[0] != 100:
            seq_x_mark = seq_x_mark.astype(np.int64)
            if x_train.shape[0] > 100:
                target = seq_x_mark[-100:, :]
#                 print(22, target.shape)
                history = history[-100:, :]
#                 print(23, history.shape)
            else:
#                 print('before pad', x_train.shape[0])
#                 print(x_train)
#                 print('before pad', seq_x_mark.shape)
#                 print(seq_x_mark.shape)
#                 print(x_train.shape[0])
                target = np.pad(seq_x_mark, pad_width=((100-x_train.shape[0], 0), (0, 0)), mode='constant')
#                 target = np.pad(seq_x_mark, (0, 100-x_train.shape[0]))
#                 print(24, target.shape)
#                 print('before pad', history.shape)
                history = np.pad(history, pad_width=((100-x_train.shape[0], 0), (0, 0)), mode='constant')
#                 history = np.pad(history, (0, 100-x_train.shape[0]))
#                 print(25, history.shape)
                
            history = np.repeat(np.expand_dims(history, axis=0), self.horizon, axis=0)[:, :, 0] #убрать expand_dims
#             history = np.repeat(history, self.horizon, axis=0)[:, 0]
#             history = np.expand_dims(history, axis=0)
#             print('aaa', history.shape)
#             print(history)
#             ts = target # 
#             print(target.shape)
            ts = np.repeat(np.expand_dims(target, axis=0), self.horizon, axis=0)
#             print(26, ts.shape)

        else:
            ts = np.repeat(np.expand_dims(seq_x_mark, axis=0), self.horizon, axis=1).astype(np.int64)
#             ts = seq_x_mark.astype(np.int64)
#             print(27, ts.shape)
            history = history.astype(np.float32)
#             print(28, history.shape)

#         print(ts.shape)
            
        task = np.full((self.horizon, ), 1).astype(np.int32)
#         task = 1
#         task = task.astype(np.int64)
#         task = np.int64(1)
#         task = 1
#         target_ts = np.expand_dims(seq_y_mark[-self.horizon:, :], axis=1).astype(np.int64)
#         print(seq_y_mark.shape)
#         target_ts = seq_y_mark[-self.horizon:, :].astype(np.int64)
#         target_ts = seq_y_mark.astype(np.int64)
        target_ts = seq_y_mark[:self.horizon, :].astype(np.int64)
#         print(29, target_ts.shape)
#         print(seq_y_mark.shape)
#         print('ts', ts.shape)
#         print('history', history.shape)
#         print('target_ts', target_ts.shape)
#         print('task', task.shape)
        model_input = {'ts': ts, 'history': history, 'target_ts': target_ts, 'task': task}
#         print(history.shape)

        return model_input, mean, std, x_test
        
    
def getDivisors(n, res=None) : 
    res = res or []
    i = 1
    while i <= n : 
        if (n % i==0) : 
            res.append(i), 
        i = i + 1
    return res

def get_closest_split(n, close_to=1440):
    all_divisors = getDivisors(n)
    for ix, val in enumerate(all_divisors):
        if close_to < val:
            if ix == 0: return val
            if (val-close_to)>(close_to - all_divisors[ix-1]):
                return all_divisors[ix-1]
            return val

def test(model, loader, device):
    trues = []
    preds = []
    with torch.no_grad():
        for batch in tqdm(loader):
            model_input = batch[0]
#             {key: value for key, value in model_input.items(): einsum(value)}
#             print(model_input['ts'].shape)
            model_input.update((k, rearrange(v, 'b h ... -> (b h) ...')) for k,v in model_input.items())
            mean = batch[1]
            std = batch[2]
            true_vals = batch[3].numpy()
            pred_vals = model(model_input) #.data.cpu()
            scaled_vals = pred_vals['result'].cpu().numpy().T.reshape(-1) * pred_vals['scale'].cpu().numpy().reshape(-1)
#             print(pred_vals['scale'])
#             print(pred_vals['result'])
            if len(loader.dataset) % loader.batch_size == 0:
                b = min(loader.batch_size, scaled_vals.shape[0])
            else:
                b = get_closest_split(scaled_vals.shape[0], loader.batch_size)
#             print(scaled_vals.shape)
            scaled_vals = rearrange(scaled_vals, '(b h) -> b h', b=loader.dataset.horizon)
#             print(std.shape)
#             print(scaled_vals.shape)
            scaled_vals_ = scaled_vals.copy()
#             std = torch.unsqueeze(std, dim=-1).repeat(1, loader.dataset.horizon).numpy()
#             mean = torch.unsqueeze(mean, dim=-1).repeat(1, loader.dataset.horizon).numpy()
            std = np.tile(np.expand_dims(std.numpy(), axis=-1), (1, loader.dataset.horizon))
            mean = np.tile(np.expand_dims(mean.numpy(), axis=-1), (1, loader.dataset.horizon))
            scaled_vals = std * scaled_vals.T + mean
#             print(scaled_vals.shape)
#             print(true_vals.shape)

            trues.append(list(true_vals))
            preds.append(list(scaled_vals))
        
#         обратно в rearrange
              
    trues = np.vstack(trues)
    preds = np.vstack(preds)
#     mae = np.abs(np.flatten())
#     return preds, trues, scaled_vals, std, mean
    return trues, preds # true_vals, scaled_vals, std, mean, pred_vals


libgomp: Invalid value for environment variable OMP_NUM_THREADS

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [2]:
pretrained = tf.keras.models.load_model("saved_weights/", custom_objects={'smape': smape})
config = {}
PATH = 'academic_data'

#     horizons
# horizons = [96, 192, 336, 720]
horizons = [96]
history = 36
for horizon in horizons:
    config[f'Weather_{horizon}_{history}'] = {
        'horizon': horizon, 
        'history': history, 
        'real_size': history, 
        'train': f'{PATH}/weather/weather_.csv', 
        'borders': None, 
        'max_size': 500,
        'flag': 'supervised'
    }
#     config[f'Traffic_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/traffic_agg_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
#     config[f'ETTh1_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTh1_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
#     config[f'ETTh2_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTh2_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
#     config[f'ETTm1_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTm1_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
#     config[f'ETTm2_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTm2_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
#     config[f'ECL_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/electricity/electricity_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }
# horizons = [24, 36, 48, 60] 
# for horizon in horizons:
#     config[f'ILI_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/illness/national_illness_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'supervised'
#     }

# #     histories
# horizon = 96
# histories = [48, 96, 192, 384, 768, 1536, 3072, 6144]
# for history in histories:
#     config[f'Weather_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/weather_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
#     config[f'ECL_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/electricity/electricity_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
# histories = [24, 48, 96, 192, 384, 768, 1536, 3072]
# for history in histories:
#     config[f'ETTh1_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTh1_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
#     config[f'ETTh2_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTh2_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
# histories = [96, 192, 384, 768, 1536, 3072, 6144, 12288]
# for history in histories:
#     config[f'ETTm1_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTm1_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
#     config[f'ETTm2_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/ETTm2_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
# histories = [336, 672, 1344, 2688, 5376, 10752, 21504, 43008]
# for history in histories:       
#     config[f'Traffic_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/weather/traffic_agg_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }
# histories = [52, 104, 156, 208, 260, 312, 364, 416] 
# horizon = 24
# for history in histories:
#     config[f'ILI_{horizon}_{history}'] = {
#         'horizon': horizon, 
#         'history': history, 
#         'real_size': history, 
#         'train': f'{PATH}/illness/national_illness_.csv', 
#         'borders': None, 
#         'max_size': 500,
#         'flag': 'few-shot'
#     }

for name, conf in config.items():
    print(config)
    start_time = time.time()
    device = 'cuda'
#         print({k: conf[k] for k in set(list(conf.keys())) - set('flag')})
    dataset = BenchZeroShotDataset(**{k: conf[k] for k in set(list(conf.keys())) - set({'flag'})})
    loader = DataLoader(
        dataset,
        batch_size=512,
        shuffle=False,
        num_workers=4,
        drop_last=False
    )
    print(len(loader))
    print(conf)
    t = test(pretrained, loader, device)
    print("--- %s seconds ---" % (time.time() - start_time))
    true = t[0]
    preds = t[1]
    nan_ratio = (len(preds.flatten()) - np.count_nonzero(np.isnan(preds))) / len(preds.flatten())
    nan_count = np.count_nonzero(np.isnan(preds))
    mae = np.nanmean(np.abs(true.flatten() - preds.flatten()))
    mse = np.nanmean(((true.flatten() - preds.flatten())**2))
    data = {'mse': mse, 'mae': mae, 'dataset': name, 'flag': conf['flag'], 'nan_ratio': nan_ratio, 'nan_count': nan_count}
    print(data)
    pd.DataFrame.from_dict(data).to_csv(f'/exps/{name}.csv')

2024-01-28 04:31:19.489781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 04:31:29.570060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78934 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:45:00.0, compute capability: 8.0
2024-01-28 04:31:29.571982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78934 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:4b:00.0, compute capability: 8.0


{'Weather_96_36': {'horizon': 96, 'history': 36, 'real_size': 36, 'train': 'academic_data/weather/weather_.csv', 'borders': None, 'max_size': 500, 'flag': 'supervised'}}
449
{'horizon': 96, 'history': 36, 'real_size': 36, 'train': 'academic_data/weather/weather_.csv', 'borders': None, 'max_size': 500, 'flag': 'supervised'}


  0%|          | 0/449 [00:00<?, ?it/s]2024-01-28 04:31:49.821494: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-28 04:31:50.425888: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:30] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_ILLEGAL_ADDRESS'

2024-01-28 04:31:50.425917: W tensorflow/core/framework/op_kernel.cc:1733] INTERNAL: 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'
2024-01-28 04:31:50.425930: W tensorflow/core/framework/op_kernel.cc:1733] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'
  0%|          | 0/449 [00:15<?, ?it/s]


InternalError: Exception encountered when calling layer "transformer_model" (type TransformerModel).

Graph execution error:

Detected at node 'transformer_block/transformer_block_ff1/Gelu/Erf' defined at (most recent call last):
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/jovyan/anaconda3/envs/fpfn/lib/python3.9/s

In [2]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
start_time = time.time()
SEQ_LEN = 48
PATH = 'academic_data'
    # configs = { 
    #     'Weather_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/weather/weather.csv', 'borders': None, 'max_size':500}, 
    #     #'Traffic_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/traffic/traffic.csv', 'borders': None, 'max_size':500}, 
    #     #'Electricity_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/electricity/electricity.csv', 'borders': None, 'max_size':500}, 
    #     'ILI_24_148': {'horizon': 24, 'history': 148, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}, 
    #     'ETTh1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh1.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTh2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh2.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTm1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm1.csv', 'borders': [34560, 46080, 57600], 'max_size':500}, 
    #     'ETTm2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm2.csv', 'borders': [34560, 46080, 57600], 'max_size':500},
    #     'Demand': {'horizon': 30, 'history': 50, 'real_size': SEQ_LEN, 'train': '/media/ssd-3t/Kostromina/pyboost_experiments/data/demand/demand_scaled_transformer_format.csv', 'borders': None, 'max_size':500, 'scale': False}, 
    # }
# 
# config = {'horizon': 24, 'history': SEQ_LEN, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh2_.csv', 'borders': None, 'max_size':500}
config = {'horizon': 96, 'history': SEQ_LEN, 'real_size': SEQ_LEN, 'train': f'{PATH}/weather/weather_.csv', 'borders': None, 'max_size':500}
# config = {'horizon': 24, 'history': SEQ_LEN, 'real_size': SEQ_LEN, 'train': f'{PATH}/traffic/traffic_agg_.csv', 'borders': None, 'max_size':500}
# config = {'horizon': 24, 'history': SEQ_LEN, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm1_.csv', 'borders': None, 'max_size':500}
# config = {'horizon': 24, 'history': SEQ_LEN, 'real_size': SEQ_LEN, 'train': f'{PATH}/electricity/electricity_.csv', 'borders': None, 'max_size':500}

device = 'cuda'
print(config)
dataset = BenchZeroShotDataset(**config)
loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=False,
    num_workers=4,
    drop_last=False,
#     collate_fn = lambda x: x
)
print(len(loader))
pretrained = tf.keras.models.load_model("saved_weights/", custom_objects={'smape': smape})
t = test(pretrained, loader, device)
print("--- %s seconds ---" % (time.time() - start_time))

{'horizon': 96, 'history': 48, 'real_size': 48, 'train': 'academic_data/weather/weather_.csv', 'borders': None, 'max_size': 500}
1796


2024-01-28 13:42:51.855169: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 13:43:03.375523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78934 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:45:00.0, compute capability: 8.0
2024-01-28 13:43:03.377557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78934 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:4b:00.0, compute capability: 8.0
  0%|          | 0/1796 [00:00<?, ?it/s]

Instructions for updating:
Use tf.identity instead.


2024-01-28 13:43:20.748915: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|██████████| 1796/1796 [10:13<00:00,  2.93it/s]


--- 633.4417746067047 seconds ---


In [4]:
true = t[0]
preds = t[1]

In [6]:
print((len(preds.flatten()) - np.count_nonzero(np.isnan(preds))) / len(preds.flatten()))
print(np.count_nonzero(np.isnan(preds)))

333888

In [7]:
np.nanmean(np.abs(true.flatten() - preds.flatten()))

0.17335314

In [8]:
np.nanmean(((true.flatten() - preds.flatten())**2))

0.05487443

In [256]:
(preds - true).sum().flatten()**2 / (preds.shape[0] * preds.shape[1])

array([1020.09424], dtype=float32)

In [259]:
preds - true

array([[ 7.60838926e-01,  7.85576701e-01,  7.73220956e-01,
         7.85795331e-01,  6.05906308e-01,  5.76660752e-01,
         6.55338168e-01,  1.63231611e-01,  1.83085680e-01,
         1.61823750e-01, -1.01785636e+00, -9.90324736e-01,
        -1.07987142e+00, -1.03580856e+00, -1.01456857e+00,
        -1.09737325e+00, -1.24257779e+00, -3.42013359e-01,
        -1.25595665e+00, -1.00601220e+00, -9.36494589e-01,
        -8.47717285e-01, -2.54504085e-01, -4.90479469e-01],
       [ 7.51732588e-01,  7.39376843e-01,  7.51951218e-01,
         5.72062194e-01,  5.42816639e-01,  6.21494055e-01,
         1.29387498e-01,  1.49241567e-01,  1.27979636e-01,
        -1.05170047e+00, -1.02416885e+00, -1.11371553e+00,
        -1.06965268e+00, -1.04841268e+00, -1.13121736e+00,
        -1.27642190e+00, -3.75857472e-01, -1.28980076e+00,
        -1.03985631e+00, -9.70338702e-01, -8.81561399e-01,
        -2.88348198e-01, -5.24323583e-01, -1.15914261e+00],
       [ 7.26241648e-01,  7.38816023e-01,  5.58927000e

In [236]:
mean.shape

torch.Size([166])

In [237]:
std = np.tile(np.expand_dims(std, axis=-1), (1, 24))
mean = np.tile(np.expand_dims(mean, axis=-1), (1, 24))

In [238]:
std * before_scale.T + mean

array([[3.762708  , 3.762708  , 3.762708  , 3.762708  , 3.762708  ,
        3.762708  , 3.762708  , 3.762708  , 3.762708  , 3.762708  ,
        3.762708  , 3.762708  , 3.762708  , 3.762708  , 3.762708  ,
        3.762708  , 3.762708  , 3.762708  , 3.762708  , 3.762708  ,
        3.762708  , 3.762708  , 3.762708  , 3.762708  ],
       [3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 ,
        3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 ,
        3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 ,
        3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 ,
        3.1334994 , 3.1334994 , 3.1334994 , 3.1334994 ],
       [4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 ,
        4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 ,
        4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 ,
        4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 ,
        4.6450367 , 4.6450367 , 4.6450367 , 4.6450367 ],
       [3.046

In [208]:
preds

array([[3.762708  , 3.1334994 , 4.6450367 , 3.0460432 , 3.4308586 ,
        1.5733851 , 1.8826025 , 3.7232633 , 3.1165524 , 4.6366754 ,
        3.068349  , 3.400432  , 1.568312  , 1.8778738 , 3.6839485 ,
        3.0950673 , 4.6260943 , 3.10868   , 3.3875377 , 1.5602854 ,
        1.8657886 , 3.6361232 , 3.0702825 , 4.606347  , 3.1493485 ,
        3.3789566 , 1.5484691 , 1.8495088 , 3.5815632 , 3.0430808 ,
        4.5812306 , 3.189567  , 3.375031  , 1.5337054 , 1.8328974 ,
        3.5118494 , 3.0069408 , 4.543119  , 3.1888347 , 3.340568  ,
        1.5187047 , 1.8172938 , 3.4394722 , 2.9744678 , 4.5088854 ,
        3.1884406 , 3.3175347 , 1.5054176 , 1.8043505 , 3.3601773 ,
        2.9430034 , 4.479875  , 3.1842723 , 3.3001025 , 1.4927809 ,
        1.7928619 , 3.2753613 , 2.9077475 , 4.450257  , 3.1781328 ,
        3.2774608 , 1.4771559 , 1.7798119 , 3.182505  , 2.8686152 ,
        4.42288   , 3.1729503 , 3.2683117 , 1.4534084 , 1.7574608 ,
        3.0663364 , 2.8130789 , 4.375531  , 3.16

In [71]:
SEQ_LEN = 36
PATH = 'academic_data'
config = {'horizon': 24, 'history': 36, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}
data = BenchZeroShotDataset(**config)[0][0]
# print(data)
# pretrained(data)
# data

In [170]:
SEQ_LEN = 36
PATH = 'academic_data'
    # configs = { 
    #     'Weather_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/weather/weather.csv', 'borders': None, 'max_size':500}, 
    #     #'Traffic_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/traffic/traffic.csv', 'borders': None, 'max_size':500}, 
    #     #'Electricity_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/electricity/electricity.csv', 'borders': None, 'max_size':500}, 
    #     'ILI_24_148': {'horizon': 24, 'history': 148, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}, 
    #     'ETTh1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh1.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTh2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh2.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTm1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm1.csv', 'borders': [34560, 46080, 57600], 'max_size':500}, 
    #     'ETTm2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm2.csv', 'borders': [34560, 46080, 57600], 'max_size':500},
    #     'Demand': {'horizon': 30, 'history': 50, 'real_size': SEQ_LEN, 'train': '/media/ssd-3t/Kostromina/pyboost_experiments/data/demand/demand_scaled_transformer_format.csv', 'borders': None, 'max_size':500, 'scale': False}, 
    # }
config = {'horizon': 24, 'history': 36, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}
device = 'cuda'
dataset = BenchZeroShotDataset(**config)
loader = DataLoader(
    dataset,
    batch_size=512,
    shuffle=False,
    num_workers=4,
    drop_last=False,
#     collate_fn = lambda x: x
)
print(len(loader))
pretrained = tf.keras.models.load_model("saved_weights/", custom_objects={'smape': smape})
test(pretrained, loader, device)

3


 33%|███▎      | 1/3 [00:02<00:04,  2.05s/it]

(12288,)
tensor([1.0285, 1.0157, 0.9774, 1.2227, 1.2192, 0.4432, 0.4567, 1.0470, 1.0353,
        1.0049, 1.2422, 1.2443, 0.4638, 0.4666, 1.0671, 1.0563, 1.0304, 1.2583,
        1.2671, 0.4791, 0.4807, 1.0866, 1.0752, 1.0519, 1.2766, 1.2905, 0.4924,
        0.4864, 1.1047, 1.0923, 1.0684, 1.2929, 1.3115, 0.5054, 0.4945, 1.1185,
        1.1049, 1.0754, 1.3043, 1.3269, 0.5138, 0.4984, 1.1263, 1.1098, 1.0674,
        1.3139, 1.3375, 0.5217, 0.5037, 1.1214, 1.0987, 1.0353, 1.3240, 1.3410,
        0.5288, 0.5100, 1.1196, 1.0963, 1.0154, 1.3316, 1.3412, 0.5226, 0.5086,
        1.1164, 1.0925, 1.0025, 1.3348, 1.3374, 0.5166, 0.5041, 1.0998, 1.0763,
        0.9794, 1.3197, 1.3191, 0.5070, 0.4968, 1.0707, 1.0527, 0.9412, 1.2819,
        1.2839, 0.5269, 0.5231, 1.0068, 1.0007, 0.8877, 1.1956, 1.2159, 0.5486,
        0.5457, 0.8973, 0.8998, 0.8083, 1.0428, 1.0873, 0.5714, 0.5703, 0.7793,
        0.7861, 0.7401, 0.8906, 0.9545, 0.5965, 0.5938, 0.6584, 0.6530, 0.6704,
        0.7386, 0.8062, 0.6147,

 67%|██████▋   | 2/3 [00:02<00:01,  1.07s/it]

(12288,)
tensor([0.4586, 1.1232, 0.6158, 0.8549, 0.8200, 0.8129, 0.4838, 0.5285, 1.2594,
        0.6926, 0.9636, 0.8387, 0.8273, 0.5970, 0.6612, 1.4477, 0.7472, 1.1048,
        0.8550, 0.8291, 0.6570, 0.7298, 1.5450, 0.7679, 1.2088, 0.8734, 0.8398,
        0.6916, 0.7649, 1.5809, 0.7982, 1.2771, 0.8915, 0.8587, 0.7323, 0.8048,
        1.6168, 0.8691, 1.3488, 0.9060, 0.8734, 0.7888, 0.8638, 1.6664, 0.9715,
        1.4408, 0.9209, 0.8819, 0.8657, 0.9495, 1.7266, 1.1527, 1.5808, 0.9308,
        0.8892, 0.9586, 1.0623, 1.8301, 1.4557, 1.8128, 0.9348, 0.9029, 1.0429,
        1.1665, 1.9228, 1.6753, 2.0015, 0.9350, 0.9099, 1.1042, 1.2438, 1.9806,
        1.8065, 2.1294, 0.9300, 0.9009, 1.1414, 1.2783, 1.9900, 1.8764, 2.1915,
        0.9237, 0.8911, 1.1653, 1.2975, 1.9838, 1.9146, 2.2241, 0.9111, 0.8786,
        1.1766, 1.3018, 1.9653, 1.9287, 2.2351, 0.8991, 0.8575, 1.1682, 1.2867,
        1.9270, 1.9154, 2.2151, 0.8848, 0.8488, 1.1462, 1.2608, 1.8735, 1.8848,
        2.1723, 0.8635, 0.8261,

100%|██████████| 3/3 [00:03<00:00,  1.03it/s]

(3984,)
tensor([3.1272, 2.9374, 3.4908, 0.4941, 1.2205, 1.6029, 1.7853, 3.1762, 2.9561,
        3.5012, 0.4545, 1.2532, 1.6095, 1.7915, 3.2233, 2.9784, 3.5140, 0.4090,
        1.2602, 1.6200, 1.8074, 3.2773, 3.0038, 3.5377, 0.3479, 1.2647, 1.6352,
        1.8283, 3.3352, 3.0307, 3.5671, 0.2632, 1.2662, 1.6532, 1.8493, 3.4001,
        3.0634, 3.6081, 0.2632, 1.2597, 1.6705, 1.8677, 3.4624, 3.0914, 3.6425,
        0.2630, 1.2528, 1.6845, 1.8817, 3.5228, 3.1163, 3.6687, 0.2619, 1.2462,
        1.6968, 1.8931, 3.5799, 3.1415, 3.6928, 0.2594, 1.2354, 1.7107, 1.9052,
        3.6336, 3.1676, 3.7133, 0.2566, 1.2304, 1.7295, 1.9239, 3.6816, 3.1984,
        3.7444, 0.2523, 1.2217, 1.7531, 1.9513, 3.7226, 3.2320, 3.7858, 0.2507,
        1.2169, 1.7792, 1.9825, 3.7519, 3.2638, 3.8322, 0.2506, 1.2059, 1.8073,
        2.0165, 3.7609, 3.2954, 3.8799, 0.2487, 1.2077, 1.8354, 2.0501, 3.7683,
        3.3247, 3.9273, 0.2473, 1.1970, 1.8595, 2.0813, 3.7572, 3.3377, 3.9664,
        0.2463, 1.1882, 1.8649, 

100%|██████████| 3/3 [00:03<00:00,  1.16s/it]


([], [])

In [60]:
SEQ_LEN = 36
PATH = 'academic_data'
config = {
    'horizon': 24, 
    'history': 36, 
    'real_size': SEQ_LEN, 
    'train': f'{PATH}/illness/national_illness.csv', 
    'borders': None, 
    'max_size':500
}
dataset = BenchZeroShotDataset(**config)
dataset[0][0]['ts'].shape

1 0
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 1.2077127
16 0.45672745
17 (36, 1)
18 -1.652356
19 0.25504655
20 -1.3972095417022705
21 (36, 1)
before pad 36
before pad (36, 5)
24 (100, 5)
before pad (36, 1)
25 (100, 1)
26 (24, 100, 5)
29 (24, 1, 5)


(24, 100, 5)

In [37]:
A = tf.ones([4, 4])

In [38]:
B = np.ones([4, 4])

In [50]:
np.pad(B, pad_width=((5, 0), (0, 0)), mode='constant')

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [43]:
tf.pad(A, [[5, 0], [0, 0]])

<tf.Tensor: shape=(9, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [77]:
SEQ_LEN = 36
PATH = 'academic_data'
config = {
    'horizon': 24, 
    'history': 36, 
    'real_size': SEQ_LEN, 
    'train': f'{PATH}/illness/national_illness.csv', 
    'borders': None, 
    'max_size':500
}
dataset = BenchZeroShotDataset(**config)
dataset[0][0]['target_ts'].shape

(24, 5)

In [78]:
SEQ_LEN = 36
PATH = 'academic_data'
config = {
    'horizon': 24, 
    'history': 36, 
    'real_size': SEQ_LEN, 
    'train': f'{PATH}/illness/national_illness.csv', 
    'borders': None, 
    'max_size':500
}
dataset = BenchZeroShotDataset(**config)
dataset[0][0]['history'].shape

(100,)

In [80]:
SEQ_LEN = 36
PATH = 'academic_data'
config = {
    'horizon': 24, 
    'history': 36, 
    'real_size': SEQ_LEN, 
    'train': f'{PATH}/illness/national_illness.csv', 
    'borders': None, 
    'max_size':500
}
dataset = BenchZeroShotDataset(**config)
dataset[0][0]['task']

1

In [56]:
SEQ_LEN = 36
PATH = 'academic_data'
    # configs = { 
    #     'Weather_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/weather/weather.csv', 'borders': None, 'max_size':500}, 
    #     #'Traffic_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/traffic/traffic.csv', 'borders': None, 'max_size':500}, 
    #     #'Electricity_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/electricity/electricity.csv', 'borders': None, 'max_size':500}, 
    #     'ILI_24_148': {'horizon': 24, 'history': 148, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}, 
    #     'ETTh1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh1.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTh2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTh2.csv', 'borders': [8640, 11520, 14400], 'max_size':500}, 
    #     'ETTm1_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm1.csv', 'borders': [34560, 46080, 57600], 'max_size':500}, 
    #     'ETTm2_96_250': {'horizon': 96, 'history': 512, 'real_size': SEQ_LEN, 'train': f'{PATH}/ETT-small/ETTm2.csv', 'borders': [34560, 46080, 57600], 'max_size':500},
    #     'Demand': {'horizon': 30, 'history': 50, 'real_size': SEQ_LEN, 'train': '/media/ssd-3t/Kostromina/pyboost_experiments/data/demand/demand_scaled_transformer_format.csv', 'borders': None, 'max_size':500, 'scale': False}, 
    # }
config = {'horizon': 24, 'history': 36, 'real_size': SEQ_LEN, 'train': f'{PATH}/illness/national_illness.csv', 'borders': None, 'max_size':500}
device = 'cuda'
dataset = BenchZeroShotDataset(**config)
loader = DataLoader(
    dataset,
    batch_size=512,
    shuffle=False,
    num_workers=4,
    drop_last=False
)
pretrained = tf.keras.models.load_model("saved_weights/", custom_objects={'smape': smape})
test(pretrained, loader, device)

  0%|          | 0/1 [00:00<?, ?it/s]

1 0
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 1.2077127
16 0.45672745
17 (36, 1)
18 -1.652356
19 0.25504655
20 -1.3972095417022705
21 (36, 1)
before pad 36
[[1.6652609 ]
 [0.8950005 ]
 [1.6956694 ]
 [1.5431633 ]
 [1.3854281 ]
 [1.0756415 ]
 [0.90813017]
 [1.2717689 ]
 [1.4466825 ]
 [1.536946  ]
 [1.7145966 ]
 [1.728557  ]
 [1.7843547 ]
 [1.6977199 ]
 [1.6790464 ]
 [1.6600492 ]
 [1.5640055 ]
 [1.361967  ]
 [1.644305  ]
 [1.5369723 ]
 [1.4711055 ]
 [1.3474602 ]
 [1.3067464 ]
 [1.3747687 ]
 [1.2564663 ]
 [1.1751786 ]
 [1.0249505 ]
 [0.79884744]
 [0.5384709 ]
 [0.67017853]
 [0.6083471 ]
 [0.5890177 ]
 [0.3851123 ]
 [0.2770057 ]
 [0.46162984]
 [0.39710516]]
before pad (36, 5)
[[2002    1    1    2    1]
 [2002    1    8    2    8]
 [2002    1   15    2   15]
 [2002    1   22    2   22]
 [2002    1   29    2   29]
 [2002    2    5    2   36]
 [2002    2   12    2   43]
 [2002    2   19    2   50]
 

 [2002   10   29    2  302]]
24 (100, 5)
before pad (36, 1)
25 (100, 1)
26 (24, 100, 5)
29 (24, 1, 5)
1 9
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 1.0325186
16 0.5040817
17 (36, 1)
18 -0.6344387
19 0.4698728
20 -0.16446589102745057
21 (36, 1)
before pad 36
[[1.536946  ]
 [1.7145966 ]
 [1.728557  ]
 [1.7843547 ]
 [1.6977199 ]
 [1.6790464 ]
 [1.6600492 ]
 [1.5640055 ]
 [1.361967  ]
 [1.644305  ]
 [1.5369723 ]
 [1.4711055 ]
 [1.3474602 ]
 [1.3067464 ]
 [1.3747687 ]
 [1.2564663 ]
 [1.1751786 ]
 [1.0249505 ]
 [0.79884744]
 [0.5384709 ]
 [0.67017853]
 [0.6083471 ]
 [0.5890177 ]
 [0.3851123 ]
 [0.2770057 ]
 [0.46162984]
 [0.39710516]
 [0.44687372]
 [0.42213592]
 [0.4344917 ]
 [0.4219173 ]
 [0.60180634]
 [0.6310519 ]
 [0.5523745 ]
 [1.044481  ]
 [1.024627  ]]
before pad (36, 5)
[[2002    3    5    2   64]
 [2002    3   12    2   71]
 [2002    3   19    2   78]
 [2002    3   26    2   85]
 [2002    4

 [2.287584  ]]
before pad (36, 5)
[[2002    4    2    2   92]
 [2002    4    9    2   99]
 [2002    4   16    2  106]
 [2002    4   23    2  113]
 [2002    4   30    2  120]
 [2002    5    7    2  127]
 [2002    5   14    2  134]
 [2002    5   21    2  141]
 [2002    5   28    2  148]
 [2002    6    4    2  155]
 [2002    6   11    2  162]
 [2002    6   18    2  169]
 [2002    6   25    2  176]
 [2002    7    2    2  183]
 [2002    7    9    2  190]
 [2002    7   16    2  197]
 [2002    7   23    2  204]
 [2002    7   30    2  211]
 [2002    8    6    2  218]
 [2002    8   13    2  225]
 [2002    8   20    2  232]
 [2002    8   27    2  239]
 [2002    9    3    2  246]
 [2002    9   10    2  253]
 [2002    9   17    2  260]
 [2002    9   24    2  267]
 [2002   10    1    2  274]
 [2002   10    8    2  281]
 [2002   10   15    2  288]
 [2002   10   22    2  295]
 [2002   10   29    2  302]
 [2002   11    5    2  309]
 [2002   11   12    2  316]
 [2002   11   19    2  323]
 [2002   11   

 [2002   12   31    2  365]]
24 (100, 5)
before pad (36, 1)
25 (100, 1)
26 (24, 100, 5)
29 (24, 1, 5)
1 18
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 1.1380947
16 0.67634255
17 (36, 1)
18 1.5351987
19 0.42821074
20 1.963509423828125
21 (36, 1)
before pad 36
[[1.644305  ]
 [1.5369723 ]
 [1.4711055 ]
 [1.3474602 ]
 [1.3067464 ]
 [1.3747687 ]
 [1.2564663 ]
 [1.1751786 ]
 [1.0249505 ]
 [0.79884744]
 [0.5384709 ]
 [0.67017853]
 [0.6083471 ]
 [0.5890177 ]
 [0.3851123 ]
 [0.2770057 ]
 [0.46162984]
 [0.39710516]
 [0.44687372]
 [0.42213592]
 [0.4344917 ]
 [0.4219173 ]
 [0.60180634]
 [0.6310519 ]
 [0.5523745 ]
 [1.044481  ]
 [1.024627  ]
 [1.0458889 ]
 [2.225569  ]
 [2.1980374 ]
 [2.287584  ]
 [2.2435212 ]
 [2.2222812 ]
 [2.305086  ]
 [2.4502904 ]
 [1.549726  ]]
before pad (36, 5)
[[2002    5    7    2  127]
 [2002    5   14    2  134]
 [2002    5   21    2  141]
 [2002    5   28    2  148]
 [2002    6 

...

 [2005    8   30    2  242]]
24 (100, 5)
before pad (36, 1)
25 (100, 1)
26 (24, 100, 5)
29 (24, 1, 5)
1 157
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 3.2444615
16 1.2168714
17 (36, 1)
18 0.27546707
19 0.14892796
20 0.42449502477645873
21 (36, 1)
before pad 36
[[4.3332973 ]
 [3.4161448 ]
 [4.294993  ]
 [4.218768  ]
 [4.068636  ]
 [3.6936471 ]
 [4.0784955 ]
 [4.3651357 ]
 [4.3514986 ]
 [4.3851385 ]
 [4.730864  ]
 [4.712313  ]
 [4.5471582 ]
 [4.10834   ]
 [4.2518744 ]
 [4.352714  ]
 [4.071854  ]
 [2.513534  ]
 [1.5720984 ]
 [1.1222665 ]
 [0.98413616]
 [0.9448609 ]
 [1.1223977 ]
 [1.2986273 ]
 [1.4162738 ]
 [1.5417643 ]
 [2.4748228 ]
 [2.571767  ]
 [2.8031168 ]
 [2.9760585 ]
 [3.2932734 ]
 [3.4688952 ]
 [3.469494  ]
 [3.787072  ]
 [3.6934023 ]
 [3.76588   ]]
before pad (36, 5)
[[2005    1    4    2    4]
 [2005    1   11    2   11]
 [2005    1   18    2   18]
 [2005    1   25    2   25]
 [2005   

 [3.717257  ]]
before pad (36, 5)
[[2005    2    1    2   32]
 [2005    2    8    2   39]
 [2005    2   15    2   46]
 [2005    2   22    2   53]
 [2005    3    1    2   60]
 [2005    3    8    2   67]
 [2005    3   15    2   74]
 [2005    3   22    2   81]
 [2005    3   29    2   88]
 [2005    4    5    2   95]
 [2005    4   12    2  102]
 [2005    4   19    2  109]
 [2005    4   26    2  116]
 [2005    5    3    2  123]
 [2005    5   10    2  130]
 [2005    5   17    2  137]
 [2005    5   24    2  144]
 [2005    5   31    2  151]
 [2005    6    7    2  158]
 [2005    6   14    2  165]
 [2005    6   21    2  172]
 [2005    6   28    2  179]
 [2005    7    5    2  186]
 [2005    7   12    2  193]
 [2005    7   19    2  200]
 [2005    7   26    2  207]
 [2005    8    2    2  214]
 [2005    8    9    2  221]
 [2005    8   16    2  228]
 [2005    8   23    2  235]
 [2005    8   30    2  242]
 [2005    9    6    2  249]
 [2005    9   13    2  256]
 [2005    9   20    2  263]
 [2005    9   

 [2005   11    1    2  305]]
24 (100, 5)
before pad (36, 1)
25 (100, 1)
26 (24, 100, 5)
29 (24, 1, 5)
1 166
2 0
(170, 36)
3 (36,)
(170, 24)
4 (24,)
5 (36, 5)
6 (24, 5)
7 ()
8 ()
9 (36,)
10 (24,)
11 (36, 5)
12 (24, 1, 5)
13 (36, 1)
14 (36, 1)
15 3.1554594
16 1.1538556
17 (36, 1)
18 0.5186432
19 0.085047565
20 0.6037907434463501
21 (36, 1)
before pad 36
[[4.3851385 ]
 [4.730864  ]
 [4.712313  ]
 [4.5471582 ]
 [4.10834   ]
 [4.2518744 ]
 [4.352714  ]
 [4.071854  ]
 [2.513534  ]
 [1.5720984 ]
 [1.1222665 ]
 [0.98413616]
 [0.9448609 ]
 [1.1223977 ]
 [1.2986273 ]
 [1.4162738 ]
 [1.5417643 ]
 [2.4748228 ]
 [2.571767  ]
 [2.8031168 ]
 [2.9760585 ]
 [3.2932734 ]
 [3.4688952 ]
 [3.469494  ]
 [3.787072  ]
 [3.6934023 ]
 [3.76588   ]
 [3.7292848 ]
 [3.670138  ]
 [3.6937215 ]
 [3.717257  ]
 [3.7478142 ]
 [3.6835475 ]
 [3.6421473 ]
 [3.783583  ]
 [3.9490438 ]]
before pad (36, 5)
[[2005    3    8    2   67]
 [2005    3   15    2   74]
 [2005    3   22    2   81]
 [2005    3   29    2   88]
 [2005    

  0%|          | 0/1 [00:10<?, ?it/s]


ValueError: Exception encountered when calling layer "transformer_model" (type TransformerModel).

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * {'history': <tf.Tensor 'x:0' shape=(170, 100, 1) dtype=float32>,
 'target_ts': <tf.Tensor 'x_1:0' shape=(170, 24, 1, 5) dtype=int64>,
 'task': <tf.Tensor 'x_2:0' shape=(170,) dtype=int64>,
 'ts': <tf.Tensor 'x_3:0' shape=(170, 24, 100, 5) dtype=int64>}
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (2 total):
    * {'history': TensorSpec(shape=(None, 100), dtype=tf.float32, name='x/history'),
 'target_ts': TensorSpec(shape=(None, 1, 5), dtype=tf.int64, name='x/target_ts'),
 'task': TensorSpec(shape=(None,), dtype=tf.int32, name='x/task'),
 'ts': TensorSpec(shape=(None, 100, 5), dtype=tf.int64, name='x/ts')}
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments 

In [ ]:
%debug

In [23]:
dataset1 = Dataset_Custom(root_path='/home/jovyan/kkuvshinova/ForecastPFN_accelerate/academic_data/ETT-small', flag='test', timeenc=1)

In [16]:
dataset1.seq_len

384

In [24]:
dataset1.__dict__

{'seq_len': 384,
 'label_len': 96,
 'pred_len': 96,
 'set_type': 2,
 'features': 'S',
 'target': 'OT',
 'scale': True,
 'timeenc': 1,
 'freq': 'h',
 'scaler': StandardScaler(),
 'train_budget': None,
 'root_path': '/home/jovyan/kkuvshinova/ForecastPFN_accelerate/academic_data/ETT-small',
 'data_path': 'ETTh1_.csv',
 'data_stamp_original':                       date  Unnamed: 0    HUFL   HULL   MUFL   MULL   LUFL  \
 13552  2018-01-16 16:00:00       13552   9.578  2.009  5.899  0.533  3.686   
 13553  2018-01-16 17:00:00       13553  12.257  2.679  7.995  0.817  4.447   
 13554  2018-01-16 18:00:00       13554  11.855  2.210  7.569  0.746  4.295   
 13555  2018-01-16 19:00:00       13555  11.588  2.076  7.427  0.569  4.142   
 13556  2018-01-16 20:00:00       13556  10.784  2.210  6.752  0.569  4.112   
 ...                    ...         ...     ...    ...    ...    ...    ...   
 17415  2018-06-26 15:00:00       17415  -1.674  3.550 -5.615  2.132  3.472   
 17416  2018-06-26 16:00:00 

In [32]:
import tensorflow as tf
tf.convert_to_tensor(tf.ones([5, 1], tf.int32).unsqueeze(0).repeat(5, 1, 1), dtype=tf.int64)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'unsqueeze'

In [35]:
tf.ones([5, 1], tf.float64).unsqueeze(0)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'unsqueeze'

In [25]:
data_loader = DataLoader(
    dataset1,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False)

In [26]:
next(iter(data_loader))

[tensor([[[-1.4631],
          [-1.5220],
          [-1.5220],
          [-1.4126],
          [-1.3873],
          [-1.3788],
          [-1.3030],
          [-1.5305],
          [-1.6064],
          [-1.5389],
          [-1.5137],
          [-1.3620],
          [-1.2693],
          [-1.2271],
          [-1.1597],
          [-1.2356],
          [-1.2946],
          [-1.1850],
          [-1.1177],
          [-1.1008],
          [-1.2102],
          [-1.3198],
          [-1.3620],
          [-1.2609],
          [-1.1766],
          [-1.2524],
          [-1.3451],
          [-1.3704],
          [-1.3282],
          [-1.2777],
          [-1.3451],
          [-1.3535],
          [-1.4041],
          [-1.3873],
          [-1.4041],
          [-1.4293],
          [-1.4631],
          [-1.4378],
          [-1.4884],
          [-1.4378],
          [-1.4378],
          [-1.4378],
          [-1.4547],
          [-1.4378],
          [-1.4968],
          [-1.4378],
          [-1.4126],
          [-1

In [9]:
def parse():

    parser = argparse.ArgumentParser(
        description='Comparing performance of ForecastPFN to other Time Series Benchmarks')

    parser.add_argument('--is_training', type=int, default=0, help='status')
    parser.add_argument('--use_gpu', type=bool, default=True, help='status')
    parser.add_argument('--itr', type=int, default=5, help='status')

    # model settings
    parser.add_argument('--model', type=str, default='ForecastPFN',
                        help='model name, options: [ForecastPFN, FEDformer, Autoformer, Informer, Transformer, Arima, Prophet]')

    # forecasting task
    parser.add_argument('--seq_len', type=int, default=384,
                        help='input sequence length')
    parser.add_argument('--label_len', type=int,
                        default=96, help='start token length')
    parser.add_argument('--pred_len', type=int, default=96,
                        help='prediction sequence length')

    # parser.add_argument('--time_budget', type=int,
    #                     help='amount of time budget to train the model')
    # parser.add_argument('--train_budget', type=int,
    #                     help='length of training sequence')

    # data loader
    parser.add_argument('--data', type=str,
                        default='ETTh1', help='dataset type')
    parser.add_argument('--root_path', type=str,
                        default='../academic_data/ETT-small/', help='root path of the data file')
    parser.add_argument('--data_path', type=str,
                        default='ETTh1_.csv', help='data file')
    parser.add_argument('--target', type=str,
                        default='OT', help='name of target column')
    parser.add_argument('--scale', type=bool, default=True,
                        help='scale the time series with sklearn.StandardScale()')

    # ForecastPFN
    parser.add_argument('--model_path', type=str, default='../saved_weights/',
                        help='encoder input size')
    parser.add_argument('--scaler', type=str, default='standard',
                        help='scale the test series with sklearn.StandardScale()')

    # Metalearn
    parser.add_argument('--metalearn_freq', type=str,
                        help='which type of model should be used for the Metalearn model. Typically M, W, or D.')
    return parser

In [10]:
parser = parse()
args = parser.parse_args()
args = resolve_args(args)

usage: ipykernel_launcher.py [-h] [--is_training IS_TRAINING]
                             [--use_gpu USE_GPU] [--itr ITR] [--model MODEL]
                             [--seq_len SEQ_LEN] [--label_len LABEL_LEN]
                             [--pred_len PRED_LEN] [--data DATA]
                             [--root_path ROOT_PATH] [--data_path DATA_PATH]
                             [--target TARGET] [--scale SCALE]
                             [--model_path MODEL_PATH] [--scaler SCALER]
                             [--metalearn_freq METALEARN_FREQ]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"dc6ad997-58c5-44b8-ac65-6167014f18e9" --shell=9007 --transport="tcp" --iopub=9009 --f=/home/jovyan/.local/share/jupyter/runtime/kernel-v2-347175Fvwj4DSEoA6W.json


SystemExit: 2

In [ ]:
data_set, data_loader = data_provider()

In [2]:
dataset1 = Dataset_Custom(root_path='/home/jovyan/kkuvshinova/ForecastPFN_accelerate/academic_data/ETT-small')
a = dataset1[0]

In [3]:
dataset1.seq_len

384

In [4]:
dataset1.label_len

96

In [5]:
dataset1.pred_len

96

In [6]:
dataset1.train_budget

In [7]:
a[0]

array([[1.70525639],
       [1.37657355],
       [1.37657355],
       [1.04801043],
       [0.67716404],
       [0.58445245],
       [0.77826037],
       [0.82042369],
       [0.64350517],
       [0.13790359],
       [0.44131252],
       [0.45808198],
       [0.34860092],
       [0.27277873],
       [0.39064452],
       [0.12101441],
       [0.38225979],
       [0.34009648],
       [0.52551944],
       [0.2895482 ],
       [0.18845187],
       [0.20534128],
       [0.20534128],
       [0.41603838],
       [0.57606772],
       [0.40753393],
       [0.44969725],
       [0.5339044 ],
       [0.60972659],
       [0.71094263],
       [0.47497139],
       [0.57606772],
       [0.51713471],
       [0.2475046 ],
       [0.55917831],
       [0.37387506],
       [0.54240885],
       [0.46658666],
       [0.60134186],
       [0.68566849],
       [0.52551944],
       [1.09855848],
       [1.14910676],
       [1.38495828],
       [1.31764054],
       [0.71094263],
       [0.67716404],
       [1.317

In [6]:
data

array([[[-1.65172522, -1.65172522, -1.65172522, ..., -1.65172522,
         -1.65172522, -1.65172522]],

       [[-1.64493755, -1.64493755, -1.64493755, ..., -1.64493755,
         -1.64493755, -1.64493755]],

       [[-1.63885194, -1.63885194, -1.63885194, ..., -1.63885194,
         -1.63885194, -1.63885194]],

       ...,

       [[-0.77096854, -0.77096854, -0.77096854, ..., -0.77096854,
         -0.77096854, -0.77096854]],

       [[-0.77120478, -0.77120478, -0.77120478, ..., -0.77120478,
         -0.77120478, -0.77120478]],

       [[-0.77096854, -0.77096854, -0.77096854, ..., -0.77096854,
         -0.77096854, -0.77096854]]])

In [7]:
np.isnan(data).sum()

0

In [8]:
np.isinf(data).sum()

0

In [9]:
data = np.load('/home/jovyan/kkuvshinova/ForecastPFN_accelerate/benchmark/results_history/ETTh1/ForecastPFN_ETTh1_sl36_ll18_pl14_timebudget_None_trainbudget_50_model-path_saved_weights_targetOT_itr_0/true.npy')

In [10]:
np.isnan(data).sum()

0

In [11]:
np.isinf(data).sum()

0

In [14]:
data = np.load('/home/jovyan/kkuvshinova/ForecastPFN_accelerate/benchmark/results_history/ETTh1/ForecastPFN_ETTh1_sl36_ll18_pl14_timebudget_None_trainbudget_50_model-path_saved_weights_targetOT_itr_0/metrics.npy', allow_pickle=True)


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [15]:
data

array({'metrics': {'mae': 0.227779650408567, 'mse': 0.09279294257578616, 'rmse': 0.3046193404493322, 'mape': 0.3487395024324892, 'mspe': 4.013795182933621}, 'train_timer': None, 'vali_timer': None, 'test_timer': 0.09956669807434082, 'args': Namespace(is_training=0, use_gpu=True, itr=5, model='ForecastPFN', seq_len=36, label_len=18, pred_len=14, time_budget=None, train_budget=50, data='ETTh1', root_path='../academic_data/ETT-small/', data_path='ETTh1_.csv', target='OT', scale=True, model_path='../saved_weights/', scaler=StandardScaler(), metalearn_freq=None, features='S', freq='h', checkpoints='./checkpoints/', embed='timeF', batch_size=512, gpu=0, use_multi_gpu=True, devices='0,1', num_workers=10, model_name='saved_weights', dvices='0,1', device_ids=[0, 1])},
      dtype=object)

In [17]:
data = np.load('/home/jovyan/kkuvshinova/ForecastPFN_accelerate/benchmark/results_history/ETTh1/ForecastPFN_ETTh1_sl36_ll18_pl96_timebudget_None_trainbudget_1_model-path_.._targetHUFL_itr_0/pred.npy')

In [18]:
data

array([[[ 0.75126138,  0.75126138,  0.75126138, ...,  0.75126138,
          0.75126138,  0.75126138]],

       [[ 0.7011685 ,  0.7011685 ,  0.7011685 , ...,  0.7011685 ,
          0.7011685 ,  0.7011685 ]],

       [[ 0.69326071,  0.69326071,  0.69326071, ...,  0.69326071,
          0.69326071,  0.69326071]],

       ...,

       [[-0.4656374 , -0.4656374 , -0.4656374 , ..., -0.4656374 ,
         -0.4656374 , -0.4656374 ]],

       [[-0.4999147 , -0.4999147 , -0.4999147 , ..., -0.4999147 ,
         -0.4999147 , -0.4999147 ]],

       [[-0.51573465, -0.51573465, -0.51573465, ..., -0.51573465,
         -0.51573465, -0.51573465]]])

In [19]:
np.isnan(data).sum()

288

In [20]:
data = np.load('/home/jovyan/kkuvshinova/ForecastPFN_accelerate/benchmark/results_history/ETTh1/ForecastPFN_ETTh1_sl36_ll18_pl96_timebudget_None_trainbudget_1_model-path_.._targetHUFL_itr_0/true.npy')

In [21]:
data

array([[[-0.5921752 ],
        [ 0.9264881 ],
        [ 1.2850153 ],
        ...,
        [-3.3131723 ],
        [-2.9546452 ],
        [-2.3007302 ]],

       [[ 0.9264881 ],
        [ 1.2850153 ],
        [ 1.7701373 ],
        ...,
        [-2.9546452 ],
        [-2.3007302 ],
        [-1.1616936 ]],

       [[ 1.2850153 ],
        [ 1.7701373 ],
        [ 1.6647992 ],
        ...,
        [-2.3007302 ],
        [-1.1616936 ],
        [ 0.8104429 ]],

       ...,

       [[-0.27584606],
        [-0.20199916],
        [-0.13870184],
        ...,
        [-1.4358245 ],
        [-2.0369916 ],
        [-0.72931933]],

       [[-0.20199916],
        [-0.13870184],
        [-0.13870184],
        ...,
        [-2.0369916 ],
        [-0.72931933],
        [ 0.28312278]],

       [[-0.13870184],
        [-0.13870184],
        [ 0.25147408],
        ...,
        [-0.72931933],
        [ 0.28312278],
        [ 0.42026702]]], dtype=float32)